

本 Notebook 展示完全基於開源模型的中文 RAG 系統：
- 開源LLM：DeepSeek-R1-Distill / Qwen2.5-7B (中文優先)
- 開源Embedding：BGE-M3 / BGE-small-zh
- 本地推理：transformers + 4-bit / llama-cpp / Ollama
- 中文處理：專門的中文切分與繁簡轉換
- 完全離線：無需任何雲端API金鑰


In [ ]:
# Chinese Open-Source RAG System
# 中文開源檢索增強生成系統 - DeepSeek/Qwen + BGE + FAISS

## Stage 1 - Setup & GPU/VRAM Check
# 環境設置與顯存檢查

import os
import pathlib
import torch
import gc
import json
import time
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple, Union
import warnings

warnings.filterwarnings("ignore")

# === Shared Cache Bootstrap (Mandatory) ===
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "/mnt/ai/cache")
cache_paths = {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}

for k, v in cache_paths.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)

print(f"[Cache] 共享快取根目錄: {AI_CACHE_ROOT}")
print(f"[GPU] CUDA可用: {torch.cuda.is_available()}")


In [ ]:
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"[GPU] 顯存容量: {gpu_memory:.1f} GB")
    print(f"[GPU] 顯卡型號: {torch.cuda.get_device_name(0)}")

    # VRAM-based model recommendations
    if gpu_memory >= 16:
        recommended_model = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
        print(f"[建議] 顯存充足，推薦使用: {recommended_model} (FP16)")
    elif gpu_memory >= 8:
        recommended_model = "Qwen/Qwen2.5-7B-Instruct"
        print(f"[建議] 中等顯存，推薦使用: {recommended_model} (4-bit)")
    else:
        recommended_model = "THUDM/chatglm3-6b"
        print(f"[建議] 顯存有限，推薦使用: {recommended_model} (4-bit)")
else:
    recommended_model = "microsoft/DialoGPT-medium"
    print(f"[警告] 無GPU，推薦CPU模型: {recommended_model}")

# Create essential directories for this notebook
nb29_paths = {
    "docs": f"{AI_CACHE_ROOT}/nb29_cn_docs",
    "vectorstore": f"{AI_CACHE_ROOT}/vectorstores/nb29_cn_oss",
    "models_cache": f"{AI_CACHE_ROOT}/models/nb29_cn_oss",
}

for name, path in nb29_paths.items():
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)
    print(f"[目錄] {name}: {path}")


# Check and install required packages
def check_install_packages():
    """檢查並安裝必要套件 (Check and install required packages)"""
    required_packages = {
        "transformers": "transformers>=4.35.0",
        "torch": "torch>=2.0.0",
        "bitsandbytes": "bitsandbytes>=0.41.0",
        "sentence_transformers": "sentence-transformers>=2.2.0",
        "faiss": "faiss-cpu>=1.7.0",
        "opencc": "opencc>=1.1.0",
        "langchain": "langchain>=0.1.0",
        "langchain_text_splitters": "langchain-text-splitters",
        "gradio": "gradio>=4.0.0",
    }

    missing_packages = []
    for package, install_cmd in required_packages.items():
        try:
            __import__(package.replace("-", "_"))
            print(f"✅ {package} 已安裝")
        except ImportError:
            missing_packages.append(install_cmd)
            print(f"❌ {package} 未安裝")

    if missing_packages:
        print(f"\n請執行以下指令安裝缺失套件:")
        print(f"pip install {' '.join(missing_packages)}")
        return False

    print("✅ 所有必要套件已安裝")
    return True


package_check_result = check_install_packages()

In [ ]:
## Stage 2 - Backend & Model Selector
# 後端與模型選擇器

# Import essential libraries
try:
    import transformers
    from transformers import (
        AutoTokenizer,
        AutoModelForCausalLM,
        BitsAndBytesConfig,
        pipeline,
        TextStreamer,
    )

    print("✅ transformers 載入成功")
except ImportError:
    print("❌ transformers 載入失敗")

try:
    import sentence_transformers
    from sentence_transformers import SentenceTransformer

    print("✅ sentence_transformers 載入成功")
except ImportError:
    print("❌ sentence_transformers 載入失敗")

try:
    import faiss

    print("✅ faiss 載入成功")
except ImportError:
    print("❌ faiss 載入失敗")

try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    print("✅ langchain_text_splitters 載入成功")
except ImportError:
    print("❌ langchain_text_splitters 載入失敗")


In [ ]:
# Configuration for Chinese RAG system
class ChineseRAGConfig:
    """中文RAG系統配置 (Chinese RAG System Configuration)"""

    # Backend options: transformers, llama_cpp, ollama
    BACKEND = "transformers"

    # Model configurations (Chinese-first)
    MODELS = {
        "deepseek-r1-7b": {
            "id": "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
            "min_vram_gb": 6,
            "supports_4bit": True,
            "chinese_ability": "excellent",
        },
        "qwen2.5-7b": {
            "id": "Qwen/Qwen2.5-7B-Instruct",
            "min_vram_gb": 6,
            "supports_4bit": True,
            "chinese_ability": "excellent",
        },
        "chatglm3-6b": {
            "id": "THUDM/chatglm3-6b",
            "min_vram_gb": 5,
            "supports_4bit": True,
            "chinese_ability": "excellent",
        },
        "yi-6b": {
            "id": "01-ai/Yi-6B-Chat",
            "min_vram_gb": 5,
            "supports_4bit": True,
            "chinese_ability": "good",
        },
    }

    # Embedding models (Chinese-optimized)
    EMBEDDING_MODELS = {
        "bge-m3": "BAAI/bge-m3",
        "bge-small-zh": "BAAI/bge-small-zh-v1.5",
        "text2vec-base": "shibing624/text2vec-base-chinese",
    }

    # Chinese text processing
    CHINESE_SEPARATORS = ["。", "！", "？", "；", "…", "\n\n", "\n", " ", ""]
    CHUNK_SIZE = 512
    CHUNK_OVERLAP = 64

    # Retrieval settings
    TOP_K = 5
    SIMILARITY_THRESHOLD = 0.6


config = ChineseRAGConfig()


def auto_select_model() -> str:
    """根據顯存自動選擇模型 (Auto-select model based on VRAM)"""
    if not torch.cuda.is_available():
        return "qwen2.5-7b"  # Fallback for CPU

    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3

    for model_key, model_info in config.MODELS.items():
        if vram_gb >= model_info["min_vram_gb"]:
            print(f"[自動選擇] 基於 {vram_gb:.1f}GB 顯存，選擇: {model_key}")
            return model_key

    # Fallback to smallest model
    return "chatglm3-6b"


# Model selection
selected_model_key = auto_select_model()
selected_model_info = config.MODELS[selected_model_key]
model_id = selected_model_info["id"]

print(f"\n[模型配置]")
print(f"- 後端 (Backend): {config.BACKEND}")
print(f"- 模型 (Model): {model_id}")
print(f"- 中文能力 (Chinese): {selected_model_info['chinese_ability']}")
print(f"- 4-bit支援: {selected_model_info['supports_4bit']}")

In [ ]:
class LLMAdapter:
    """輕量LLM適配器 (Lightweight LLM Adapter)"""

    def __init__(
        self, model_id: str, backend: str = "transformers", load_in_4bit: bool = True
    ):
        self.model_id = model_id
        self.backend = backend
        self.load_in_4bit = load_in_4bit
        self.model = None
        self.tokenizer = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        print(f"[LLM] 初始化 {backend} 後端，模型: {model_id}")
        self._load_model()

    def _load_model(self):
        """載入模型 (Load model)"""
        try:
            if self.backend == "transformers":
                self._load_transformers()
            elif self.backend == "llama_cpp":
                self._load_llama_cpp()
            elif self.backend == "ollama":
                self._load_ollama()
            else:
                raise ValueError(f"不支援的後端: {self.backend}")

        except Exception as e:
            print(f"❌ 模型載入失敗: {e}")
            print(f"🔄 嘗試降級載入...")
            self._fallback_load()

    def _load_transformers(self):
        """載入Transformers模型 (Load Transformers model)"""
        # Configure quantization for low VRAM
        if self.load_in_4bit and torch.cuda.is_available():
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16,
            )
        else:
            bnb_config = None

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_id,
            trust_remote_code=True,
            cache_dir=cache_paths["TRANSFORMERS_CACHE"],
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            quantization_config=bnb_config,
            device_map="auto" if torch.cuda.is_available() else None,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            trust_remote_code=True,
            cache_dir=cache_paths["TRANSFORMERS_CACHE"],
        )

        print(f"✅ Transformers模型載入成功")

    def _load_llama_cpp(self):
        """載入llama-cpp模型 (Load llama-cpp model)"""
        try:
            from llama_cpp import Llama

            # This would require GGUF model files
            print("⚠️ llama-cpp後端需要GGUF模型檔案")
            raise NotImplementedError("llama-cpp backend not implemented in this demo")
        except ImportError:
            print("❌ llama-cpp-python 未安裝")
            raise

    def _load_ollama(self):
        """載入Ollama模型 (Load Ollama model)"""
        try:
            import ollama

            # This would require Ollama service running
            print("⚠️ Ollama後端需要Ollama服務運行")
            raise NotImplementedError("Ollama backend not implemented in this demo")
        except ImportError:
            print("❌ ollama 未安裝")
            raise

    def _fallback_load(self):
        """降級載入 (Fallback loading)"""
        print("🔄 嘗試CPU模式載入...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_id, trust_remote_code=True
            )
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_id,
                torch_dtype=torch.float32,
                device_map=None,
                trust_remote_code=True,
            )
            self.device = "cpu"
            print("✅ CPU模式載入成功")
        except Exception as e:
            print(f"❌ 降級載入也失敗: {e}")
            raise

    def generate(
        self,
        messages: List[Dict[str, str]],
        max_length: int = 1024,
        temperature: float = 0.7,
    ) -> str:
        """生成回應 (Generate response)"""
        if not self.model or not self.tokenizer:
            return "模型未載入"

        try:
            # Format messages for different model types
            if "qwen" in self.model_id.lower():
                # Qwen format
                formatted_text = self._format_qwen_messages(messages)
            elif "chatglm" in self.model_id.lower():
                # ChatGLM format
                formatted_text = self._format_chatglm_messages(messages)
            elif "deepseek" in self.model_id.lower():
                # DeepSeek format (similar to ChatML)
                formatted_text = self._format_deepseek_messages(messages)
            else:
                # Generic format
                formatted_text = self._format_generic_messages(messages)

            # Tokenize
            inputs = self.tokenizer(
                formatted_text, return_tensors="pt", truncation=True, max_length=2048
            ).to(self.device)

            # Generate
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=inputs.input_ids.shape[1] + max_length,
                    temperature=temperature,
                    do_sample=True,
                    top_p=0.8,
                    top_k=50,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )

            # Decode response
            response = self.tokenizer.decode(
                outputs[0][inputs.input_ids.shape[1] :], skip_special_tokens=True
            ).strip()

            return response

        except Exception as e:
            print(f"❌ 生成失敗: {e}")
            return f"生成錯誤: {str(e)}"

    def _format_qwen_messages(self, messages: List[Dict]) -> str:
        """格式化Qwen消息 (Format Qwen messages)"""
        formatted = "<|im_start|>system\n你是一個樂於助人的AI助手。<|im_end|>\n"
        for msg in messages:
            role = msg.get("role", "user")
            content = msg.get("content", "")
            formatted += f"<|im_start|>{role}\n{content}<|im_end|>\n"
        formatted += "<|im_start|>assistant\n"
        return formatted

    def _format_chatglm_messages(self, messages: List[Dict]) -> str:
        """格式化ChatGLM消息 (Format ChatGLM messages)"""
        formatted = ""
        for msg in messages:
            if msg.get("role") == "user":
                formatted += f"[Round 1]\n\n問：{msg.get('content', '')}\n\n答："
        return formatted

    def _format_deepseek_messages(self, messages: List[Dict]) -> str:
        """格式化DeepSeek消息 (Format DeepSeek messages)"""
        formatted = "<｜begin▁of▁sentence｜>"
        for msg in messages:
            role = msg.get("role", "user")
            content = msg.get("content", "")
            if role == "user":
                formatted += f"User: {content}\n\nAssistant: "
        return formatted

    def _format_generic_messages(self, messages: List[Dict]) -> str:
        """通用消息格式 (Generic message format)"""
        formatted = ""
        for msg in messages:
            role = msg.get("role", "user")
            content = msg.get("content", "")
            if role == "user":
                formatted += f"Human: {content}\n\nAssistant: "
        return formatted

In [ ]:
# Initialize LLM
print(f"\n[載入模型] 開始載入 {model_id}...")
start_time = time.time()

try:
    llm = LLMAdapter(
        model_id=model_id,
        backend=config.BACKEND,
        load_in_4bit=selected_model_info["supports_4bit"] and torch.cuda.is_available(),
    )
    load_time = time.time() - start_time
    print(f"✅ 模型載入完成，耗時: {load_time:.2f}秒")

    # Quick test
    test_messages = [{"role": "user", "content": "你好，請用中文回答：1+1等於多少？"}]
    test_response = llm.generate(test_messages, max_length=50)
    print(f"[測試回應] {test_response}")

except Exception as e:
    print(f"❌ 模型初始化失敗: {e}")
    llm = None

## Stage 2 Summary
print(
    f"""
🎯 Stage 2 完成總結:
✅ 完成項目:
- 建立LLM適配器支援多種中文開源模型
- 實現4-bit量化低顯存載入
- 支援Qwen/DeepSeek/ChatGLM等主流中文模型

🧠 核心觀念:
- 開源優先策略：完全避免雲端API依賴
- 中文模型適配：針對不同模型的提示詞格式
- 資源優化：基於顯存容量自動選擇合適模型

🚀 下一步: 建立中文文檔資料庫與嵌入系統
"""
)

In [ ]:
## Stage 3 - Data Ingest (本地中文文檔)
# 本地中文資料載入與處理


def create_sample_chinese_docs():
    """建立範例中文文檔 (Create sample Chinese documents)"""
    docs_dir = pathlib.Path(nb29_paths["docs"])

    sample_docs = {
        "AI技術發展趨勢.md": """# 人工智慧技術發展趨勢

## 大型語言模型的突破

近年來，大型語言模型（Large Language Models, LLMs）在自然語言處理領域取得了革命性突破。從GPT系列到國產的通義千問、文心一言等，這些模型展現出強大的文本理解和生成能力。

### 技術特點

1. **規模化訓練**：使用大量文本資料進行預訓練，參數量從數十億到數千億不等
2. **多模態能力**：結合文本、圖像、語音等多種模態的處理能力
3. **中文優化**：針對中文語言特性進行專門優化和訓練

### 應用領域

- 智能客服與對話系統
- 內容創作與寫作輔助
- 程式碼生成與除錯
- 教育輔導與知識問答
- 翻譯與多語言處理

## 檢索增強生成技術

檢索增強生成（Retrieval-Augmented Generation, RAG）是當前最重要的LLM應用架構之一。它結合了資訊檢索和文本生成的優勢，能夠基於外部知識庫提供準確且及時的答案。

### 核心優勢

1. **知識時效性**：可以整合最新的外部資訊
2. **領域專業性**：針對特定領域的深度知識
3. **可解釋性**：提供資訊來源和引用依據
4. **成本效益**：避免重複訓練大模型的高昂成本
""",
        "開源LLM生態系統.md": """# 開源大型語言模型生態系統

## 主要開源模型家族

### Llama系列（Meta）
- Llama 2: 7B, 13B, 70B參數版本
- Code Llama: 專門的程式碼生成模型
- Llama 3: 更強的多語言和推理能力

### 中文開源模型

#### 通義千問（Qwen）系列
- Qwen2.5: 阿里雲開源的強大中文模型
- 支援32K上下文長度
- 優秀的中文理解和生成能力
- 多種規格：0.5B到72B參數

#### DeepSeek系列
- DeepSeek-R1: 基於強化學習的推理優化
- DeepSeek-Coder: 專業的程式碼模型
- 在數學和程式設計任務上表現優異

#### ChatGLM系列（清華大學）
- ChatGLM3-6B: 輕量級中文對話模型
- 支援多輪對話和工具調用
- 針對中文語境深度優化

#### 其他重要模型
- Baichuan: 百川智能開源模型
- Yi: 零一萬物開源模型系列
- InternLM: 上海AI實驗室書生模型

## 技術生態與工具鏈

### 推理引擎
- vLLM: 高效能推理加速
- llama.cpp: 跨平台CPU推理
- Ollama: 本地模型管理平台
- TensorRT-LLM: NVIDIA GPU優化

### 量化技術
- GPTQ: 權重量化技術
- AWQ: 啟動感知量化
- GGUF: llama.cpp的量化格式
- BitsAndBytes: 動態量化載入

### 微調框架
- LoRA/QLoRA: 低秩適應微調
- PEFT: 參數效率微調庫
- DeepSpeed: 大規模分散式訓練
- Unsloth: 高效微調加速

## 部署與應用

### 本地部署方案
1. **個人電腦**: 使用量化模型和優化推理
2. **企業私有雲**: 建立內部LLM服務
3. **邊緣設備**: 輕量級模型部署

### 成本考量
- 開源模型免費使用，僅需承擔推理成本
- 避免API調用費用和資料隱私風險
- 可根據需求靈活調整模型規模
""",
        "RAG系統實作指南.md": """# RAG檢索增強生成系統實作指南

## 系統架構設計

### 核心組件

1. **文檔處理模組**
   - 文檔載入：支援PDF、Word、Markdown等格式
   - 文本切分：智能分段保持語義完整性
   - 繁簡轉換：處理繁體與簡體中文差異

2. **向量化模組**
   - 嵌入模型：BGE-M3、text2vec等中文優化模型
   - 向量資料庫：FAISS、Chroma、Pinecone等
   - 索引建立：支援大規模文檔集合

3. **檢索模組**
   - 語義搜尋：基於向量相似度的檢索
   - 重排序：使用專門的reranker模型提升精度
   - 混合檢索：結合關鍵詞和語義檢索

4. **生成模組**
   - 提示工程：設計有效的RAG提示模板
   - 上下文管理：控制檢索內容的長度和品質
   - 回答生成：確保答案的準確性和相關性

## 中文處理優化

### 分詞與切分策略
```python
# 中文分隔符優先序列
separators = ["。", "！", "？", "；", "…", "\\n\\n", "\\n", " "]

# 適合中文的chunk size設定
chunk_size = 512  # 中文字符數
overlap = 64      # 重疊字符數
```

### 嵌入模型選擇
- **BGE-M3**: 多語言模型，中英文效果佳
- **BGE-small-zh**: 專門的中文嵌入模型
- **text2vec-base-chinese**: 輕量級中文模型

### 檢索評估指標
- **Recall@K**: 前K個結果中正確答案的召回率
- **MRR**: 平均倒數排名
- **NDCG**: 標準化折扣累計增益

## 系統評估與優化

### 檢索品質評估
1. **相關性評估**: 檢索結果與查詢的相關程度
2. **覆蓋率分析**: 知識庫對查詢領域的覆蓋範圍
3. **響應時間**: 檢索和生成的延遲指標

### 生成品質評估
1. **事實準確性**: 答案是否符合檢索到的事實
2. **完整性**: 答案是否充分回應了問題
3. **流暢性**: 生成文本的自然度和可讀性
4. **引用準確性**: 是否正確引用了來源資訊

### 系統優化策略
- **快取機制**: 對常見查詢建立快取
- **批次處理**: 提升向量化處理效率
- **模型選擇**: 根據精度和效能需求選擇合適模型
- **硬體優化**: GPU加速、記憶體管理

## 實際部署考量

### 效能最佳化
- 使用量化模型減少記憶體佔用
- 實施模型並行和流水線處理
- 最佳化向量檢索演算法

### 資料安全
- 本地部署避免資料外洩
- 存取控制和使用者認證
- 敏感資訊過濾和匿名化

### 擴展性設計
- 微服務架構便於維護和擴展
- 支援動態添加新文檔和知識源
- 監控和日誌系統完善

這個指南提供了建立高品質中文RAG系統的完整框架，從技術選型到實際部署都有詳細說明。
""",
    }

    created_files = []
    for filename, content in sample_docs.items():
        file_path = docs_dir / filename
        if not file_path.exists():
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)
            created_files.append(filename)
            print(f"✅ 建立範例文檔: {filename}")

    return created_files


def scan_local_documents() -> List[Dict[str, Any]]:
    """掃描本地文檔目錄 (Scan local document directory)"""
    docs_dir = pathlib.Path(nb29_paths["docs"])

    if not any(docs_dir.iterdir()):
        print(f"📁 文檔目錄為空，建立範例文檔...")
        created_files = create_sample_chinese_docs()
        print(f"✅ 建立了 {len(created_files)} 個範例文檔")

    # Scan for documents
    supported_extensions = [".md", ".txt", ".mdx"]
    documents = []

    for file_path in docs_dir.iterdir():
        if file_path.is_file() and file_path.suffix.lower() in supported_extensions:
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()

                doc_info = {
                    "filename": file_path.name,
                    "filepath": str(file_path),
                    "content": content,
                    "size": len(content),
                    "extension": file_path.suffix,
                    "modified": datetime.fromtimestamp(
                        file_path.stat().st_mtime
                    ).isoformat(),
                }
                documents.append(doc_info)
                print(f"📄 載入文檔: {file_path.name} ({len(content)} 字符)")

            except Exception as e:
                print(f"❌ 載入失敗 {file_path.name}: {e}")

    print(f"\n📚 總共載入 {len(documents)} 個文檔")
    return documents


# Optional: Chinese text processing with OpenCC
def setup_chinese_converter():
    """設置中文繁簡轉換 (Setup Chinese Traditional/Simplified converter)"""
    try:
        import opencc

        converter = opencc.OpenCC("t2s")  # Traditional to Simplified
        print("✅ OpenCC 繁簡轉換器載入成功")
        return converter
    except ImportError:
        print("⚠️ OpenCC 未安裝，跳過繁簡轉換功能")
        return None


chinese_converter = setup_chinese_converter()


def preprocess_chinese_text(text: str, converter=None) -> str:
    """預處理中文文本 (Preprocess Chinese text)"""
    # Basic cleaning
    text = text.strip()

    # Optional: Convert traditional to simplified
    if converter:
        try:
            text = converter.convert(text)
        except:
            pass  # Ignore conversion errors

    # Remove excessive whitespace
    import re

    text = re.sub(r"\n\s*\n", "\n\n", text)  # Normalize line breaks
    text = re.sub(r" +", " ", text)  # Normalize spaces

    return text


# Text splitting with Chinese optimization
def create_chinese_text_splitter():
    """建立中文文本分割器 (Create Chinese text splitter)"""
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        separators=config.CHINESE_SEPARATORS,
        chunk_size=config.CHUNK_SIZE,
        chunk_overlap=config.CHUNK_OVERLAP,
        length_function=len,  # Use character count for Chinese
        is_separator_regex=False,
    )

    print(f"📝 中文文本分割器配置:")
    print(f"- 塊大小: {config.CHUNK_SIZE} 字符")
    print(f"- 重疊: {config.CHUNK_OVERLAP} 字符")
    print(f"- 分隔符: {config.CHINESE_SEPARATORS[:5]}...")

    return text_splitter


def process_documents_to_chunks(documents: List[Dict]) -> List[Dict[str, Any]]:
    """將文檔處理為文本塊 (Process documents into text chunks)"""
    text_splitter = create_chinese_text_splitter()
    all_chunks = []

    for doc in documents:
        print(f"\n📄 處理文檔: {doc['filename']}")

        # Preprocess text
        content = preprocess_chinese_text(doc["content"], chinese_converter)

        # Split into chunks
        chunks = text_splitter.split_text(content)

        # Create chunk metadata
        for i, chunk in enumerate(chunks):
            chunk_info = {
                "content": chunk,
                "source_file": doc["filename"],
                "source_path": doc["filepath"],
                "chunk_index": i,
                "chunk_id": f"{doc['filename']}_chunk_{i}",
                "char_count": len(chunk),
                "metadata": {
                    "filename": doc["filename"],
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "file_size": doc["size"],
                },
            }
            all_chunks.append(chunk_info)

        print(f"  ✅ 分割為 {len(chunks)} 個文本塊")

    print(f"\n📊 總計處理結果:")
    print(f"- 文檔數量: {len(documents)}")
    print(f"- 文本塊數量: {len(all_chunks)}")
    print(
        f"- 平均塊大小: {sum(c['char_count'] for c in all_chunks) / len(all_chunks):.1f} 字符"
    )

    return all_chunks


# Load and process documents
print(f"\n=== Stage 3: 載入本地中文文檔 ===")
documents = scan_local_documents()
chunks = process_documents_to_chunks(documents)

# Save chunks for debugging
chunks_file = pathlib.Path(nb29_paths["vectorstore"]) / "chunks.json"
try:
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    print(f"💾 文本塊已保存至: {chunks_file}")
except Exception as e:
    print(f"⚠️ 保存失敗: {e}")

## Stage 3 Summary
print(
    f"""
🎯 Stage 3 完成總結:
✅ 完成項目:
- 自動建立中文範例文檔（AI技術、開源LLM、RAG指南）
- 實現中文文本預處理與繁簡轉換
- 設計中文優化的文本分割策略
- 生成帶有完整元資料的文本塊

🧠 核心觀念:
- 中文分詞特性：使用標點符號作為主要分隔符
- 語義完整性：保持文本塊的語義連貫性
- 元資料追蹤：為每個文本塊保留來源資訊

🚀 下一步: 建立中文嵌入向量與FAISS索引
"""
)

In [ ]:
## Stage 4 - Embedding & Index (中文嵌入與索引)
# 中文嵌入模型與向量索引建立


def load_chinese_embedding_model(model_name: str = "bge-m3"):
    """載入中文嵌入模型 (Load Chinese embedding model)"""
    model_id = config.EMBEDDING_MODELS.get(
        model_name, config.EMBEDDING_MODELS["bge-m3"]
    )

    print(f"\n🔗 載入嵌入模型: {model_id}")

    try:
        # Load with explicit cache directory
        embedding_model = SentenceTransformer(
            model_id, cache_folder=cache_paths["HF_HOME"]
        )

        # Test embedding
        test_text = "這是一個中文測試句子。"
        test_embedding = embedding_model.encode([test_text])

        print(f"✅ 嵌入模型載入成功")
        print(f"📏 向量維度: {test_embedding.shape[1]}")
        print(f"🧪 測試嵌入: {test_embedding[0][:5]}...")

        return embedding_model

    except Exception as e:
        print(f"❌ 嵌入模型載入失敗: {e}")
        print(f"🔄 嘗試使用備用模型...")

        # Fallback to smaller model
        try:
            fallback_model_id = config.EMBEDDING_MODELS["bge-small-zh"]
            embedding_model = SentenceTransformer(fallback_model_id)
            print(f"✅ 備用模型載入成功: {fallback_model_id}")
            return embedding_model
        except Exception as e2:
            print(f"❌ 備用模型也失敗: {e2}")
            raise


def create_embeddings_for_chunks(
    chunks: List[Dict], embedding_model
) -> Tuple[List[np.ndarray], List[Dict]]:
    """為文本塊建立嵌入向量 (Create embeddings for text chunks)"""
    print(f"\n🧮 開始建立 {len(chunks)} 個文本塊的嵌入向量...")

    # Extract text content
    texts = [chunk["content"] for chunk in chunks]

    # Create embeddings in batches to manage memory
    batch_size = 32
    all_embeddings = []

    start_time = time.time()

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        batch_embeddings = embedding_model.encode(
            batch_texts,
            normalize_embeddings=True,  # Normalize for cosine similarity
            show_progress_bar=False,
        )
        all_embeddings.extend(batch_embeddings)

        if (i // batch_size + 1) % 5 == 0:
            print(
                f"  📊 進度: {min(i+batch_size, len(texts))}/{len(texts)} ({(i+batch_size)/len(texts)*100:.1f}%)"
            )

    embedding_time = time.time() - start_time
    print(f"✅ 嵌入建立完成，耗時: {embedding_time:.2f}秒")
    print(f"📈 平均速度: {len(chunks)/embedding_time:.1f} chunks/sec")

    # Convert to numpy array
    embeddings_array = np.array(all_embeddings).astype("float32")
    print(f"📏 嵌入矩陣形狀: {embeddings_array.shape}")

    return embeddings_array, chunks


def build_faiss_index(
    embeddings: np.ndarray, chunks: List[Dict]
) -> Tuple[faiss.Index, List[Dict]]:
    """建立FAISS向量索引 (Build FAISS vector index)"""
    print(f"\n🗃️ 建立FAISS索引...")

    dimension = embeddings.shape[1]
    num_vectors = embeddings.shape[0]

    print(f"📏 向量維度: {dimension}")
    print(f"📊 向量數量: {num_vectors}")

    # Choose index type based on dataset size
    if num_vectors < 1000:
        # For small datasets, use flat index (exact search)
        index = faiss.IndexFlatIP(dimension)  # Inner Product (cosine similarity)
        print("🔍 使用 Flat 索引 (精確搜索)")
    else:
        # For larger datasets, use approximate search
        nlist = min(100, num_vectors // 10)  # Number of clusters
        index = faiss.IndexIVFFlat(faiss.IndexFlatIP(dimension), dimension, nlist)
        print(f"🔍 使用 IVF 索引 (近似搜索, nlist={nlist})")

        # Train the index
        print("🏋️ 訓練索引...")
        index.train(embeddings)

    # Add vectors to index
    print("📥 添加向量到索引...")
    index.add(embeddings)

    print(f"✅ FAISS索引建立完成")
    print(f"📊 索引統計: {index.ntotal} 個向量")

    return index, chunks


def save_vector_store(
    index: faiss.Index, chunks: List[Dict], embedding_model_name: str
):
    """保存向量存儲 (Save vector store)"""
    vectorstore_dir = pathlib.Path(nb29_paths["vectorstore"])

    # Save FAISS index
    index_file = vectorstore_dir / "index.faiss"
    faiss.write_index(index, str(index_file))

    # Save chunks metadata
    chunks_file = vectorstore_dir / "chunks_metadata.json"
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    # Save configuration
    config_file = vectorstore_dir / "config.json"
    config_data = {
        "embedding_model": embedding_model_name,
        "index_type": "flat" if "Flat" in str(type(index)) else "ivf",
        "dimension": index.d,
        "total_vectors": index.ntotal,
        "created_at": datetime.now().isoformat(),
        "chunk_size": config.CHUNK_SIZE,
        "chunk_overlap": config.CHUNK_OVERLAP,
    }

    with open(config_file, "w", encoding="utf-8") as f:
        json.dump(config_data, f, indent=2)

    print(f"💾 向量存儲已保存至: {vectorstore_dir}")
    print(f"  - 索引檔案: index.faiss")
    print(f"  - 元資料: chunks_metadata.json")
    print(f"  - 配置: config.json")


# Load embedding model and create index
print(f"\n=== Stage 4: 建立中文嵌入與向量索引 ===")

embedding_model = load_chinese_embedding_model("bge-m3")
embeddings, chunk_metadata = create_embeddings_for_chunks(chunks, embedding_model)
faiss_index, indexed_chunks = build_faiss_index(embeddings, chunk_metadata)

# Save vector store
save_vector_store(faiss_index, indexed_chunks, "bge-m3")

# Memory cleanup
del embeddings
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

## Stage 4 Summary
print(
    f"""
🎯 Stage 4 完成總結:
✅ 完成項目:
- 載入中文優化的BGE-M3嵌入模型
- 批次處理文本塊建立嵌入向量
- 建立FAISS向量索引支援語義搜索
- 保存完整的向量存儲與元資料

🧠 核心觀念:
- 中文嵌入：使用專門優化的中文嵌入模型
- 向量正規化：確保餘弦相似度計算的準確性
- 索引策略：根據資料規模選擇合適的索引類型
- 記憶體管理：批次處理避免記憶體溢出

🚀 下一步: 實現檢索、重排序與生成流程
"""
)

In [ ]:
## Stage 5 - Retrieve → (Re-rank) → Generate
# 檢索、重排序與生成流程


class ChineseRAGRetriever:
    """中文RAG檢索器 (Chinese RAG Retriever)"""

    def __init__(
        self,
        faiss_index: faiss.Index,
        chunks: List[Dict],
        embedding_model,
        llm_adapter: LLMAdapter,
    ):
        self.index = faiss_index
        self.chunks = chunks
        self.embedding_model = embedding_model
        self.llm = llm_adapter
        self.reranker = None

        print(f"🔍 RAG檢索器初始化完成")
        print(f"📊 索引向量數: {self.index.ntotal}")
        print(f"📄 文本塊數: {len(self.chunks)}")

    def load_reranker(self, model_name: str = "bge-reranker-base"):
        """載入重排序模型 (Load reranker model)"""
        try:
            reranker_model_id = f"BAAI/{model_name}"
            from sentence_transformers import CrossEncoder

            self.reranker = CrossEncoder(
                reranker_model_id, cache_folder=cache_paths["HF_HOME"]
            )
            print(f"✅ 重排序模型載入成功: {model_name}")

        except Exception as e:
            print(f"⚠️ 重排序模型載入失敗: {e}")
            print(f"💡 將跳過重排序步驟")
            self.reranker = None

    def retrieve(self, query: str, top_k: int = 5) -> List[Dict]:
        """檢索相關文本塊 (Retrieve relevant text chunks)"""
        # Encode query
        query_embedding = self.embedding_model.encode(
            [query], normalize_embeddings=True
        )
        query_vector = query_embedding.astype("float32")

        # Search in FAISS index
        scores, indices = self.index.search(query_vector, top_k)

        # Prepare results
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx != -1:  # Valid index
                chunk = self.chunks[idx].copy()
                chunk["similarity_score"] = float(score)
                chunk["rank"] = i + 1
                results.append(chunk)

        return results

    def rerank(self, query: str, retrieved_chunks: List[Dict]) -> List[Dict]:
        """重排序檢索結果 (Rerank retrieved results)"""
        if not self.reranker or len(retrieved_chunks) <= 1:
            return retrieved_chunks

        try:
            # Prepare query-document pairs
            pairs = [(query, chunk["content"]) for chunk in retrieved_chunks]

            # Get reranking scores
            rerank_scores = self.reranker.predict(pairs)

            # Update chunks with rerank scores and sort
            for chunk, score in zip(retrieved_chunks, rerank_scores):
                chunk["rerank_score"] = float(score)

            # Sort by rerank score (descending)
            reranked = sorted(
                retrieved_chunks, key=lambda x: x["rerank_score"], reverse=True
            )

            # Update ranks
            for i, chunk in enumerate(reranked):
                chunk["rerank_rank"] = i + 1

            print(f"🔄 重排序完成，調整了 {len(reranked)} 個結果的順序")
            return reranked

        except Exception as e:
            print(f"❌ 重排序失敗: {e}")
            return retrieved_chunks

    def generate_answer(
        self, query: str, context_chunks: List[Dict], max_context_length: int = 2000
    ) -> Dict[str, Any]:
        """基於檢索上下文生成答案 (Generate answer based on retrieved context)"""

        # Prepare context from chunks
        context_parts = []
        total_length = 0
        used_chunks = []

        for chunk in context_chunks:
            content = chunk["content"]
            if total_length + len(content) <= max_context_length:
                context_parts.append(f"【來源：{chunk['source_file']}】\n{content}")
                total_length += len(content)
                used_chunks.append(chunk)
            else:
                break

        context = "\n\n".join(context_parts)

        # Create RAG prompt
        rag_prompt = f"""基於以下資料來源，請用中文回答問題。請確保答案準確且有根據，並在答案末尾列出參考的資料來源。

問題：{query}

參考資料：
{context}

請提供詳細且準確的回答："""

        # Generate answer
        messages = [{"role": "user", "content": rag_prompt}]

        start_time = time.time()
        answer = self.llm.generate(messages, max_length=512, temperature=0.3)
        generation_time = time.time() - start_time

        # Prepare result
        result = {
            "query": query,
            "answer": answer,
            "context_chunks": used_chunks,
            "context_length": total_length,
            "generation_time": generation_time,
            "sources": list(set(chunk["source_file"] for chunk in used_chunks)),
        }

        return result

    def answer(
        self, query: str, top_k: int = 5, use_reranker: bool = True
    ) -> Dict[str, Any]:
        """端到端問答 (End-to-end question answering)"""
        print(f"\n❓ 問題: {query}")

        # Step 1: Retrieve
        print(f"🔍 檢索中...")
        retrieved = self.retrieve(query, top_k)
        print(f"📄 檢索到 {len(retrieved)} 個相關文本塊")

        # Step 2: Rerank (optional)
        if use_reranker and self.reranker:
            print(f"🔄 重排序中...")
            retrieved = self.rerank(query, retrieved)

        # Step 3: Generate
        print(f"✍️ 生成答案中...")
        result = self.generate_answer(query, retrieved)

        print(f"✅ 回答完成 (耗時: {result['generation_time']:.2f}秒)")

        return result


# Initialize retriever
print(f"\n=== Stage 5: 建立RAG檢索生成系統 ===")

rag_retriever = ChineseRAGRetriever(
    faiss_index=faiss_index,
    chunks=indexed_chunks,
    embedding_model=embedding_model,
    llm_adapter=llm,
)

# Optional: Load reranker (comment out if VRAM is limited)
# rag_retriever.load_reranker("bge-reranker-base")

# Demo queries
demo_queries = [
    "什麼是大型語言模型？",
    "RAG系統有什麼優勢？",
    "如何選擇合適的開源中文模型？",
    "中文文本處理有什麼特殊考量？",
]

print(f"\n🎯 RAG系統演示")
print("=" * 50)

demo_results = []
for query in demo_queries[:2]:  # Limit to 2 queries for demo
    try:
        result = rag_retriever.answer(query, top_k=3, use_reranker=False)
        demo_results.append(result)

        print(f"\n📝 問答結果:")
        print(f"Q: {result['query']}")
        print(f"A: {result['answer']}")
        print(f"📚 參考來源: {', '.join(result['sources'])}")
        print(f"⏱️ 生成時間: {result['generation_time']:.2f}秒")
        print("-" * 50)

    except Exception as e:
        print(f"❌ 問答失敗: {e}")

## Stage 5 Summary
print(
    f"""
🎯 Stage 5 完成總結:
✅ 完成項目:
- 實現完整的RAG檢索生成流程
- 支援FAISS向量檢索與餘弦相似度計算
- 設計中文優化的RAG提示模板
- 提供端到端問答介面與來源追蹤

🧠 核心觀念:
- 檢索策略：基於語義相似度的向量檢索
- 上下文管理：控制輸入長度避免超出模型限制
- 提示工程：設計有效的中文RAG提示模板
- 來源歸屬：確保答案可追溯到原始文檔

🚀 下一步: 實現評估指標與效能分析
"""
)

In [ ]:
## Stage 6 - Evaluation (輕量評估)
# 檢索與生成品質評估


def create_evaluation_dataset():
    """建立評估資料集 (Create evaluation dataset)"""
    eval_queries = [
        {
            "query": "什麼是大型語言模型？",
            "expected_keywords": ["LLM", "語言模型", "GPT", "訓練", "參數"],
            "expected_sources": ["AI技術發展趨勢.md"],
            "category": "definition",
        },
        {
            "query": "RAG系統的核心優勢是什麼？",
            "expected_keywords": ["RAG", "檢索增強", "知識庫", "時效性", "專業性"],
            "expected_sources": ["AI技術發展趨勢.md", "RAG系統實作指南.md"],
            "category": "concept",
        },
        {
            "query": "有哪些重要的開源中文模型？",
            "expected_keywords": ["Qwen", "ChatGLM", "DeepSeek", "開源", "中文"],
            "expected_sources": ["開源LLM生態系統.md"],
            "category": "enumeration",
        },
        {
            "query": "如何評估RAG系統的檢索品質？",
            "expected_keywords": ["Recall", "MRR", "NDCG", "相關性", "評估"],
            "expected_sources": ["RAG系統實作指南.md"],
            "category": "methodology",
        },
        {
            "query": "中文文本分割有什麼特殊考量？",
            "expected_keywords": ["分詞", "標點符號", "語義", "chunk", "中文"],
            "expected_sources": ["RAG系統實作指南.md"],
            "category": "technical",
        },
    ]

    return eval_queries


class RAGEvaluator:
    """RAG系統評估器 (RAG System Evaluator)"""

    def __init__(self, retriever: ChineseRAGRetriever):
        self.retriever = retriever

    def evaluate_retrieval(
        self, eval_queries: List[Dict], k_values: List[int] = [1, 3, 5]
    ) -> Dict:
        """評估檢索品質 (Evaluate retrieval quality)"""
        print(f"\n📊 評估檢索品質...")

        results = {
            "recall_at_k": {k: [] for k in k_values},
            "mrr_scores": [],
            "query_results": [],
        }

        for query_data in eval_queries:
            query = query_data["query"]
            expected_sources = set(query_data["expected_sources"])

            # Retrieve documents
            retrieved = self.retriever.retrieve(query, top_k=max(k_values))
            retrieved_sources = [chunk["source_file"] for chunk in retrieved]

            # Calculate Recall@K
            query_recall = {}
            for k in k_values:
                retrieved_k = set(retrieved_sources[:k])
                relevant_found = len(retrieved_k.intersection(expected_sources))
                recall_k = (
                    relevant_found / len(expected_sources) if expected_sources else 0
                )
                query_recall[k] = recall_k
                results["recall_at_k"][k].append(recall_k)

            # Calculate MRR (Mean Reciprocal Rank)
            reciprocal_rank = 0
            for i, source in enumerate(retrieved_sources):
                if source in expected_sources:
                    reciprocal_rank = 1.0 / (i + 1)
                    break
            results["mrr_scores"].append(reciprocal_rank)

            # Store detailed results
            query_result = {
                "query": query,
                "expected_sources": list(expected_sources),
                "retrieved_sources": retrieved_sources,
                "recall_at_k": query_recall,
                "reciprocal_rank": reciprocal_rank,
            }
            results["query_results"].append(query_result)

            print(
                f"  📝 {query[:30]}... - Recall@3: {query_recall[3]:.2f}, RR: {reciprocal_rank:.2f}"
            )

        # Calculate averages
        avg_recall = {}
        for k in k_values:
            avg_recall[k] = sum(results["recall_at_k"][k]) / len(
                results["recall_at_k"][k]
            )

        avg_mrr = sum(results["mrr_scores"]) / len(results["mrr_scores"])

        results["average_recall_at_k"] = avg_recall
        results["average_mrr"] = avg_mrr

        return results

    def evaluate_generation(self, eval_queries: List[Dict]) -> Dict:
        """評估生成品質 (Evaluate generation quality)"""
        print(f"\n📝 評估生成品質...")

        results = {
            "groundedness_scores": [],
            "keyword_coverage": [],
            "response_lengths": [],
            "generation_times": [],
            "query_results": [],
        }

        for query_data in eval_queries:
            query = query_data["query"]
            expected_keywords = query_data["expected_keywords"]

            # Generate answer
            answer_result = self.retriever.answer(query, top_k=3, use_reranker=False)
            answer = answer_result["answer"]

            groundedness = self.calculate_groundedness(answer, context_text)
            results["groundedness_scores"].append(groundedness)

            # Evaluate keyword coverage
            answer_lower = answer.lower()
            keyword_hits = sum(
                1 for keyword in expected_keywords if keyword.lower() in answer_lower
            )
            coverage = keyword_hits / len(expected_keywords) if expected_keywords else 0
            results["keyword_coverage"].append(coverage)

            # Record metrics
            results["response_lengths"].append(len(answer))
            results["generation_times"].append(answer_result["generation_time"])

            query_result = {
                "query": query,
                "answer": answer,
                "expected_keywords": expected_keywords,
                "keyword_coverage": coverage,
                "groundedness": groundedness,
                "response_length": len(answer),
                "generation_time": answer_result["generation_time"],
                "sources_used": answer_result["sources"],
            }
            results["query_results"].append(query_result)

            print(
                f"  📝 {query[:30]}... - 關鍵詞覆蓋: {coverage:.2f}, 事實性: {groundedness:.2f}"
            )

        # Calculate averages
        results["average_keyword_coverage"] = sum(results["keyword_coverage"]) / len(
            results["keyword_coverage"]
        )
        results["average_groundedness"] = sum(results["groundedness_scores"]) / len(
            results["groundedness_scores"]
        )
        results["average_response_length"] = sum(results["response_lengths"]) / len(
            results["response_lengths"]
        )
        results["average_generation_time"] = sum(results["generation_times"]) / len(
            results["generation_times"]
        )

        return results

    def calculate_groundedness(self, answer: str, context: str) -> float:
        """計算答案的事實依據性 (Calculate answer groundedness)"""
        if not context.strip():
            return 0.0

        # Simple approach: check if key phrases from answer appear in context
        import re

        # Split answer into sentences
        answer_sentences = re.split(r"[。！？]", answer)
        answer_sentences = [s.strip() for s in answer_sentences if s.strip()]

        if not answer_sentences:
            return 0.0

        grounded_sentences = 0
        for sentence in answer_sentences:
            # Check if sentence has substantial overlap with context
            sentence_words = set(sentence.split())
            context_words = set(context.split())

            if len(sentence_words) > 2:  # Only check substantial sentences
                overlap = len(sentence_words.intersection(context_words))
                overlap_ratio = overlap / len(sentence_words)

                if overlap_ratio > 0.3:  # At least 30% word overlap
                    grounded_sentences += 1

        return grounded_sentences / len(answer_sentences)

    def performance_benchmark(self, num_queries: int = 10) -> Dict:
        """效能基準測試 (Performance benchmark)"""
        print(f"\n⚡ 執行效能基準測試...")

        test_query = "什麼是大型語言模型的主要特點？"

        retrieval_times = []
        generation_times = []
        total_times = []

        for i in range(num_queries):
            start_time = time.time()

            # Retrieval timing
            retrieval_start = time.time()
            retrieved = self.retriever.retrieve(test_query, top_k=5)
            retrieval_time = time.time() - retrieval_start
            retrieval_times.append(retrieval_time)

            # Generation timing
            generation_start = time.time()
            result = self.retriever.generate_answer(test_query, retrieved)
            generation_time = time.time() - generation_start
            generation_times.append(generation_time)

            total_time = time.time() - start_time
            total_times.append(total_time)

            if (i + 1) % 5 == 0:
                print(f"  📊 完成 {i+1}/{num_queries} 次測試")

        # Calculate statistics
        def calc_stats(times):
            return {
                "mean": sum(times) / len(times),
                "min": min(times),
                "max": max(times),
                "std": (
                    sum((t - sum(times) / len(times)) ** 2 for t in times) / len(times)
                )
                ** 0.5,
            }

        benchmark_results = {
            "retrieval": calc_stats(retrieval_times),
            "generation": calc_stats(generation_times),
            "total": calc_stats(total_times),
            "num_queries": num_queries,
        }

        return benchmark_results


# Run evaluation
print(f"\n=== Stage 6: RAG系統評估 ===")

eval_dataset = create_evaluation_dataset()
evaluator = RAGEvaluator(rag_retriever)

print(f"📋 評估資料集: {len(eval_dataset)} 個查詢")

# Retrieval evaluation
retrieval_results = evaluator.evaluate_retrieval(eval_dataset)

print(f"\n📊 檢索評估結果:")
for k, recall in retrieval_results["average_recall_at_k"].items():
    print(f"  - Recall@{k}: {recall:.3f}")
print(f"  - MRR: {retrieval_results['average_mrr']:.3f}")

# Generation evaluation
generation_results = evaluator.evaluate_generation(eval_dataset[:3])  # Limit for demo

print(f"\n📝 生成評估結果:")
print(f"  - 平均關鍵詞覆蓋率: {generation_results['average_keyword_coverage']:.3f}")
print(f"  - 平均事實依據性: {generation_results['average_groundedness']:.3f}")
print(f"  - 平均回應長度: {generation_results['average_response_length']:.1f} 字符")
print(f"  - 平均生成時間: {generation_results['average_generation_time']:.2f} 秒")

# Performance benchmark
performance_results = evaluator.performance_benchmark(num_queries=5)

print(f"\n⚡ 效能基準測試:")
print(
    f"  - 檢索時間: {performance_results['retrieval']['mean']:.3f}±{performance_results['retrieval']['std']:.3f}s"
)
print(
    f"  - 生成時間: {performance_results['generation']['mean']:.3f}±{performance_results['generation']['std']:.3f}s"
)
print(
    f"  - 總時間: {performance_results['total']['mean']:.3f}±{performance_results['total']['std']:.3f}s"
)

# Save evaluation results
eval_results = {
    "retrieval_evaluation": retrieval_results,
    "generation_evaluation": generation_results,
    "performance_benchmark": performance_results,
    "evaluation_timestamp": datetime.now().isoformat(),
    "model_config": {
        "llm_model": model_id,
        "embedding_model": "bge-m3",
        "chunk_size": config.CHUNK_SIZE,
        "top_k": config.TOP_K,
    },
}

eval_file = pathlib.Path(nb29_paths["vectorstore"]) / "evaluation_results.json"
try:
    with open(eval_file, "w", encoding="utf-8") as f:
        json.dump(eval_results, f, ensure_ascii=False, indent=2)
    print(f"💾 評估結果已保存至: {eval_file}")
except Exception as e:
    print(f"⚠️ 保存評估結果失敗: {e}")

## Stage 6 Summary
print(
    f"""
🎯 Stage 6 完成總結:
✅ 完成項目:
- 建立多維度RAG評估框架（檢索+生成+效能）
- 實現Recall@K、MRR等檢索指標計算
- 設計事實依據性與關鍵詞覆蓋率評估
- 執行效能基準測試與統計分析

🧠 核心觀念:
- 檢索評估：使用標準資訊檢索指標
- 生成評估：結合事實性與完整性指標
- 效能分析：分離檢索與生成的時間成本
- 持續改進：建立可重複的評估流程

🚀 下一步: 建立Gradio互動介面
"""
)

In [ ]:
## Stage 7 - (Optional) Gradio Quick Interface
# 可選：Gradio快速互動介面

try:
    import gradio as gr

    gradio_available = True
    print("✅ Gradio 可用，建立互動介面...")
except ImportError:
    gradio_available = False
    print("⚠️ Gradio 未安裝，跳過介面建立")


def create_gradio_interface():
    """建立Gradio聊天介面 (Create Gradio chat interface)"""
    if not gradio_available or not llm:
        return None

    def rag_chat(message, history):
        """RAG聊天處理函數 (RAG chat handler)"""
        try:
            # Get RAG answer
            result = rag_retriever.answer(message, top_k=3, use_reranker=False)

            # Format response with sources
            response = result["answer"]
            if result["sources"]:
                response += f"\n\n📚 **參考來源**: {', '.join(result['sources'])}"

            # Add to history
            history.append([message, response])
            return history, ""

        except Exception as e:
            error_response = f"❌ 處理失敗: {str(e)}"
            history.append([message, error_response])
            return history, ""

    # Create interface
    with gr.Blocks(title="中文RAG系統", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🇨🇳 中文開源RAG問答系統")
        gr.Markdown("基於DeepSeek/Qwen + BGE + FAISS的完全開源解決方案")

        chatbot = gr.Chatbot(label="AI助手", height=400, show_label=True)

        with gr.Row():
            msg = gr.Textbox(label="輸入問題", placeholder="請輸入您的問題...", scale=4)
            send_btn = gr.Button("發送", scale=1, variant="primary")

        # Sample questions
        gr.Markdown("### 💡 範例問題:")
        sample_questions = [
            "什麼是大型語言模型？",
            "RAG系統有什麼優勢？",
            "有哪些重要的開源中文模型？",
            "如何評估RAG系統品質？",
        ]

        for question in sample_questions:
            gr.Button(question, size="sm").click(
                lambda q=question: (chatbot.value + [[q, "處理中..."]], ""),
                outputs=[chatbot, msg],
            ).then(
                rag_chat,
                inputs=[gr.Textbox(value=question, visible=False), chatbot],
                outputs=[chatbot, msg],
            )

        # Event handlers
        msg.submit(rag_chat, inputs=[msg, chatbot], outputs=[chatbot, msg])
        send_btn.click(rag_chat, inputs=[msg, chatbot], outputs=[chatbot, msg])

        # System info
        with gr.Accordion("🔧 系統資訊", open=False):
            gr.Markdown(
                f"""
            - **LLM模型**: {model_id}
            - **嵌入模型**: BGE-M3
            - **向量數量**: {faiss_index.ntotal}
            - **文檔數量**: {len(documents)}
            - **後端**: {config.BACKEND}
            """
            )

    return demo


# Create and launch Gradio interface
if gradio_available and llm:
    print(f"\n🌐 建立Gradio介面...")
    gradio_demo = create_gradio_interface()

    # For notebook environment, use share=False and inbrowser=False
    try:
        print(f"🚀 啟動Gradio介面 (本地存取)")
        gradio_demo.launch(
            server_name="127.0.0.1",
            server_port=7860,
            share=False,
            inbrowser=False,
            quiet=True,
        )
        print(f"✅ Gradio介面已啟動: http://127.0.0.1:7860")
    except Exception as e:
        print(f"⚠️ Gradio啟動失敗: {e}")
        print(f"💡 可能需要在終端機中運行此notebook")

## Stage 7 Summary
print(
    f"""
🎯 Stage 7 完成總結:
✅ 完成項目:
- 建立Gradio互動式聊天介面
- 整合RAG問答與來源顯示功能
- 提供範例問題快速測試
- 顯示系統配置資訊

🧠 核心觀念:
- 使用者體驗：提供直觀的聊天介面
- 資訊透明：顯示答案來源與系統狀態
- 快速測試：預設範例問題便於驗證
- 本地部署：完全本地化的互動環境

🚀 下一步: 完整系統總結與優化建議
"""
)

In [ ]:
# Stage 8 - Final Summary & Optimizations
# 最終總結與優化建議


def generate_system_report():
    """生成系統完整報告 (Generate comprehensive system report)"""

    # System configuration
    system_config = {
        "llm_model": model_id,
        "embedding_model": "BAAI/bge-m3",
        "vector_store": "FAISS",
        "backend": config.BACKEND,
        "quantization": "4-bit" if torch.cuda.is_available() else "none",
        "device": "cuda" if torch.cuda.is_available() else "cpu",
    }

    # Data statistics
    data_stats = {
        "total_documents": len(documents),
        "total_chunks": len(chunks),
        "vector_dimension": faiss_index.d,
        "index_size": faiss_index.ntotal,
        "avg_chunk_size": (
            sum(c["char_count"] for c in chunks) / len(chunks) if chunks else 0
        ),
    }

    # Performance summary
    if "performance_results" in locals():
        perf_summary = {
            "avg_retrieval_time": performance_results["retrieval"]["mean"],
            "avg_generation_time": performance_results["generation"]["mean"],
            "avg_total_time": performance_results["total"]["mean"],
        }
    else:
        perf_summary = {"note": "Performance benchmark not run"}

    # Quality metrics
    if "retrieval_results" in locals() and "generation_results" in locals():
        quality_summary = {
            "recall_at_3": retrieval_results["average_recall_at_k"].get(3, 0),
            "mrr": retrieval_results["average_mrr"],
            "keyword_coverage": generation_results["average_keyword_coverage"],
            "groundedness": generation_results["average_groundedness"],
        }
    else:
        quality_summary = {"note": "Quality evaluation not run"}

    report = {
        "system_configuration": system_config,
        "data_statistics": data_stats,
        "performance_metrics": perf_summary,
        "quality_metrics": quality_summary,
        "generated_at": datetime.now().isoformat(),
    }

    return report


# Generate final report
print(f"\n=== 🎯 中文開源RAG系統完整報告 ===")

final_report = generate_system_report()

print(f"\n📋 系統配置:")
for key, value in final_report["system_configuration"].items():
    print(f"  - {key}: {value}")

print(f"\n📊 資料統計:")
for key, value in final_report["data_statistics"].items():
    print(f"  - {key}: {value}")

if "note" not in final_report["performance_metrics"]:
    print(f"\n⚡ 效能指標:")
    for key, value in final_report["performance_metrics"].items():
        print(f"  - {key}: {value:.3f}s")

if "note" not in final_report["quality_metrics"]:
    print(f"\n🎯 品質指標:")
    for key, value in final_report["quality_metrics"].items():
        print(f"  - {key}: {value:.3f}")

# Save final report
report_file = pathlib.Path(nb29_paths["vectorstore"]) / "system_report.json"
try:
    with open(report_file, "w", encoding="utf-8") as f:
        json.dump(final_report, f, ensure_ascii=False, indent=2)
    print(f"\n💾 完整報告已保存至: {report_file}")
except Exception as e:
    print(f"⚠️ 保存報告失敗: {e}")

In [ ]:
## Smoke Test & Validation
# 煙霧測試與驗證


def run_comprehensive_smoke_test():
    """執行完整的煙霧測試 (Run comprehensive smoke test)"""
    print(f"\n🧪 執行系統煙霧測試...")

    tests = {
        "shared_cache_setup": False,
        "model_loading": False,
        "embedding_model": False,
        "vector_index": False,
        "retrieval_function": False,
        "generation_function": False,
        "end_to_end_qa": False,
    }

    try:
        # Test 1: Shared cache setup
        tests["shared_cache_setup"] = all(
            pathlib.Path(path).exists() for path in nb29_paths.values()
        )

        # Test 2: Model loading
        tests["model_loading"] = llm is not None and llm.model is not None

        # Test 3: Embedding model
        tests["embedding_model"] = embedding_model is not None

        # Test 4: Vector index
        tests["vector_index"] = faiss_index is not None and faiss_index.ntotal > 0

        # Test 5: Retrieval function
        try:
            test_retrieval = rag_retriever.retrieve("測試", top_k=1)
            tests["retrieval_function"] = len(test_retrieval) > 0
        except:
            tests["retrieval_function"] = False

        # Test 6: Generation function
        try:
            test_messages = [{"role": "user", "content": "你好"}]
            test_response = llm.generate(test_messages, max_length=10)
            tests["generation_function"] = len(test_response) > 0
        except:
            tests["generation_function"] = False

        # Test 7: End-to-end QA
        try:
            test_result = rag_retriever.answer(
                "什麼是AI？", top_k=2, use_reranker=False
            )
            tests["end_to_end_qa"] = (
                "answer" in test_result and len(test_result["answer"]) > 0
            )
        except:
            tests["end_to_end_qa"] = False

    except Exception as e:
        print(f"❌ 測試執行失敗: {e}")

    # Report results
    print(f"\n📊 煙霧測試結果:")
    for test_name, passed in tests.items():
        status = "✅ PASS" if passed else "❌ FAIL"
        print(f"  - {test_name}: {status}")

    overall_health = sum(tests.values()) / len(tests)
    health_status = (
        "🟢 優秀"
        if overall_health >= 0.9
        else "🟡 良好" if overall_health >= 0.7 else "🔴 需要改進"
    )

    print(f"\n🏥 系統整體健康度: {overall_health:.1%} {health_status}")

    return tests


# Run smoke test
smoke_test_results = run_comprehensive_smoke_test()

# Optimization recommendations
print(
    f"""
🚀 系統優化建議:
===============

💡 效能優化:
- 使用GGUF量化模型進一步減少記憶體使用
- 實現向量檢索結果快取機制
- 考慮使用vLLM或TensorRT-LLM加速推理
- 批次處理多個查詢提升吞吐量

📚 資料品質:
- 增加更多領域文檔豐富知識庫
- 實現增量文檔更新機制
- 優化中文文本切分策略
- 建立文檔品質評估流程

🔧 功能擴展:
- 增加多模態支援（圖片、表格）
- 實現對話歷史記憶功能
- 添加即時網路搜尋補充
- 建立使用者反饋收集機制

🛡️ 穩定性提升:
- 添加更完善的錯誤處理與重試機制
- 實現模型熱切換功能
- 建立系統監控與告警
- 優化記憶體管理避免洩漏

📈 擴展部署:
- 容器化部署（Docker）
- 微服務架構拆分
- 負載均衡與高可用性
- API介面標準化
"""
)

print(
    f"""
✅ 中文開源RAG系統建立完成！

🎯 關鍵成就:
- 完全基於開源模型的中文RAG系統
- 支援DeepSeek/Qwen等主流中文模型
- 實現從文檔載入到問答的完整流程
- 提供評估指標與效能分析
- 建立互動式聊天介面

📚 系統能力:
- 🤖 中文LLM: {model_id}
- 🔍 向量檢索: FAISS + BGE-M3
- 📄 文檔處理: {len(documents)} 個文檔，{len(chunks)} 個文本塊
- ⚡ 即時問答: 平均 {final_report.get('performance_metrics', {}).get('avg_total_time', 'N/A')} 秒響應
- 🌐 Web介面: Gradio互動介面

🚀 可立即使用於:
- 企業內部知識問答
- 技術文檔查詢系統
- 教育輔助工具
- 研究資料分析

💡 完全離線運行，保護資料隱私！
"""
)

In [ ]:
class LLMAdapter:
    def __init__(
        self, model_id: str, backend: str = "transformers", load_in_4bit: bool = True
    ):
        # 支援多種中文模型格式的統一介面
        self._load_model()

    def _format_qwen_messages(self, messages: List[Dict]) -> str:
        # Qwen專用的ChatML格式
        formatted = "<|im_start|>system\n你是一個樂於助人的AI助手。<|im_end|>\n"
        # ... 處理訊息格式

In [ ]:
# 中文分隔符優先序列
CHINESE_SEPARATORS = ["。", "！", "？", "；", "…", "\n\n", "\n", " "]


def create_chinese_text_splitter():
    return RecursiveCharacterTextSplitter(
        separators=CHINESE_SEPARATORS,
        chunk_size=512,  # 中文字符數
        chunk_overlap=64,
        length_function=len,
    )

In [ ]:
class ChineseRAGRetriever:
    def answer(self, query: str, top_k: int = 5) -> Dict[str, Any]:
        # 1. 向量檢索
        retrieved = self.retrieve(query, top_k)

        # 2. 可選重排序
        if self.reranker:
            retrieved = self.rerank(query, retrieved)

        # 3. 生成答案
        return self.generate_answer(query, retrieved)

In [ ]:
# === 中文開源RAG系統煙霧測試 ===
def run_comprehensive_smoke_test():
    tests = {
        "shared_cache_setup": False,
        "model_loading": False,
        "embedding_model": False,
        "vector_index": False,
        "retrieval_function": False,
        "generation_function": False,
        "end_to_end_qa": False,
    }

    try:
        # 測試共享快取
        tests["shared_cache_setup"] = all(
            pathlib.Path(path).exists() for path in nb29_paths.values()
        )

        # 測試模型載入
        tests["model_loading"] = llm is not None and llm.model is not None

        # 測試端到端問答
        test_result = rag_retriever.answer("什麼是AI？", top_k=2)
        tests["end_to_end_qa"] = len(test_result["answer"]) > 0

        # ... 其他測試

    except Exception as e:
        print(f"❌ 測試失敗: {e}")

    overall_health = sum(tests.values()) / len(tests)
    print(f"🏥 系統健康度: {overall_health:.1%}")
    return tests


smoke_test_results = run_comprehensive_smoke_test()


## 6. 本章小結

### ✅ 完成項目
* **完全開源的中文RAG系統** (Fully Open-Source Chinese RAG)：無需任何雲端API，支援DeepSeek/Qwen/ChatGLM等主流中文模型
* **中文優化的文本處理** (Chinese-Optimized Text Processing)：專門的中文分詞、繁簡轉換與語義切分策略
* **高效能向量檢索** (High-Performance Vector Retrieval)：BGE-M3嵌入 + FAISS索引，支援大規模中文文檔檢索
* **多維度評估體系** (Multi-Dimensional Evaluation)：包含檢索品質(Recall@K, MRR)、生成品質(事實性、完整性)與效能指標
* **互動式Web介面** (Interactive Web Interface)：Gradio聊天介面支援即時問答與來源追蹤

### 🧠 核心概念與原理要點
* **開源優先策略** (Open-Source First Strategy)：完全避免對商業API的依賴，確保資料隱私與成本可控
* **中文語言特性** (Chinese Language Characteristics)：針對中文標點符號、語義結構與繁簡差異的專門處理
* **模型量化技術** (Model Quantization)：使用4-bit量化大幅降低顯存需求，讓7B模型在8GB顯卡上運行
* **向量語義檢索** (Vector Semantic Retrieval)：基於BGE-M3的中文語義理解，超越關鍵詞匹配的檢索精度
* **端到端評估** (End-to-End Evaluation)：從檢索到生成的完整品質評估，確保系統可靠性

### ⚠️ 常見問題與注意事項
* **模型相容性**：不同中文模型的提示詞格式差異需要專門適配
* **顯存管理**：大型模型載入需要合理的量化策略與記憶體清理
* **中文編碼**：確保UTF-8編碼一致性，避免亂碼問題
* **檢索精度**：向量檢索可能遺漏關鍵詞精確匹配，需要混合檢索策略
* **回答品質**：開源模型的生成品質可能不如商業模型，需要更精細的提示工程

### 🚀 下一步優化建議

1. **混合檢索架構** (Hybrid Retrieval Architecture)：結合BM25關鍵詞檢索與向量語義檢索，提升檢索召回率
2. **模型蒸餾優化** (Model Distillation)：使用教師-學生架構進一步壓縮模型大小
3. **增量學習機制** (Incremental Learning)：支援動態添加新文檔而無需重建整個索引
4. **多輪對話記憶** (Multi-turn Conversation Memory)：整合對話歷史提升上下文理解
5. **領域適應微調** (Domain Adaptation Fine-tuning)：針對特定領域使用LoRA微調提升專業性

---

**🎯 關鍵成就**: 建立了完全基於開源技術的中文RAG系統，實現從文檔處理到問答生成的端到端流程，無需任何商業API依賴

**📚 準備就緒**: 具備了部署生產級中文知識問答系統的完整技術棧，可應用於企業內部知識管理、教育輔助、技術支援等場景

**💡 核心價值**: 
- **隱私保護**: 完全本地運行，企業敏感資料不會外洩
- **成本可控**: 僅需一次性硬體投資，無持續API費用
- **自主可控**: 基於開源技術，避免供應商鎖定風險
- **中文優化**: 專門針對中文語言特性設計，效果優於通用方案

這個替代版本完全滿足了您的要求：中文優先、開源LLM優先、嚴格遵守共享模型快取、提供完整的MVP與評估體系。可以與原nb29並存，為不同需求場景提供選擇。